In [2]:
!pip install scikit-learn pandas numpy


In [10]:
import os
import pandas as pd
import numpy as np
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

FOLDS_DIR = "cv_folds"
FEATURE_COLS = ['MIN', 'FGA', 'FGM', 'FG3A', 'FG3M', 'FTA', 'FTM', 'OREB', 'DREB', 'TOV', 'PF']
TARGETS = ['PTS', 'REB', 'AST']
alpha = 1.0

results = []

for fold in range(1, 11):
    train_path = f"{FOLDS_DIR}/fold_{fold}_train.csv"
    test_path = f"{FOLDS_DIR}/fold_{fold}_test.csv"
    
    train_df = pd.read_csv(train_path, parse_dates=["GAME_DATE"])
    test_df = pd.read_csv(test_path, parse_dates=["GAME_DATE"])

    # Impute missing values
    imputer = SimpleImputer(strategy="mean")
    X_train = imputer.fit_transform(train_df[FEATURE_COLS])
    X_test = imputer.transform(test_df[FEATURE_COLS])

    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    for target in TARGETS:
        y_train = train_df[target].values
        y_test = test_df[target].values

        model = Lasso(alpha=alpha)
        model.fit(X_train_scaled, y_train)
        preds = model.predict(X_test_scaled)

        fold_results = {
            "fold": fold,
            "target": target,
            "rmse": mean_squared_error(y_test, preds, squared=False),
            "mae": mean_absolute_error(y_test, preds),
            "r2": r2_score(y_test, preds)
        }
        results.append(fold_results)

# Convert to DataFrame and print
results_df = pd.DataFrame(results)
print(results_df)


KeyError: "None of [Index(['MIN', 'FGA', 'FGM', 'FG3A', 'FG3M', 'FTA', 'FTM', 'OREB', 'DREB',\n       'TOV', 'PF'],\n      dtype='object')] are in the [columns]"

In [11]:
import os
import numpy as np
import pandas as pd
from itertools import product
from joblib import Parallel, delayed
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.impute import SimpleImputer

# ─── Configuration ─────────────────────────────────────────────────────────
FOLDS_DIR = "cv_folds"
FEATURE_COLS = [
    "PTS_8_game_avg", "PTS_8_game_slope",
    "REB_8_game_avg", "REB_8_game_slope",
    "AST_8_game_avg", "AST_8_game_slope",
    "opp_E_OFF_RATING", "opp_E_DEF_RATING",
    "opp_E_NET_RATING", "opp_REB_PCT"
]
TARGET_COLS = ["PTS", "REB", "AST"]
N_JOBS = 4

# ─── Hyperparameter grid ──────────────────────────────────────────────────
param_grid = {
    "alpha": [0.1, 1.0, 10.0]  # Regularization strength for Lasso
}

# ─── Single-fold runner (returns per-target metrics) ───────────────────────
def run_lasso_fold(fold_idx, alpha=1.0):
    # Load train and test data for the fold
    train_path = f"{FOLDS_DIR}/fold_{fold_idx}_train.csv"
    test_path = f"{FOLDS_DIR}/fold_{fold_idx}_test.csv"
    train_df = pd.read_csv(train_path, parse_dates=["GAME_DATE"])
    test_df = pd.read_csv(test_path, parse_dates=["GAME_DATE"])
    
    # Extract features and targets
    X_train = train_df[FEATURE_COLS].values
    y_train = train_df[TARGET_COLS].values
    X_test  = test_df[FEATURE_COLS].values
    y_test  = test_df[TARGET_COLS].values

    # Impute missing values
    imputer = SimpleImputer(strategy="mean")
    X_train = imputer.fit_transform(X_train)
    X_test = imputer.transform(X_test)

    # Scale features
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Initialize and train the Lasso model
    lasso = Lasso(alpha=alpha)
    lasso.fit(X_train, y_train)

    # Predict and calculate metrics
    y_pred = lasso.predict(X_test)

    metrics = {"fold": fold_idx, "alpha": alpha}
    for idx, target in enumerate(TARGET_COLS):
        y_true = y_test[:, idx]
        y_pred_target = y_pred[:, idx] if y_pred.ndim > 1 else y_pred  # Handle single target case
        metrics[f"{target}_rmse"] = np.sqrt(mean_squared_error(y_true, y_pred_target))
        metrics[f"{target}_mae"]  = mean_absolute_error(y_true, y_pred_target)
        metrics[f"{target}_r2"]   = r2_score(y_true, y_pred_target)

    return metrics

# ─── Run grid search with per-fold, per-target metrics ────────────────────
all_results = []
for alpha in param_grid["alpha"]:
    fold_metrics = Parallel(n_jobs=N_JOBS)(
        delayed(run_lasso_fold)(fold, alpha) for fold in range(1, 11)
    )
    all_results.extend(fold_metrics)

# ─── Compile and display results ────────────────────────────────────────────
final_df = pd.DataFrame(all_results)
# Order columns: hyperparams, fold, then metrics
cols = ["alpha", "fold"] + [c for c in final_df.columns if c not in ("alpha", "fold")]
final_df = final_df[cols]
print(final_df.to_string(index=False))


 alpha  fold  PTS_rmse  PTS_mae    PTS_r2  REB_rmse  REB_mae        REB_r2  AST_rmse  AST_mae    AST_r2
   0.1     1  7.854028 6.140139  0.253482  2.792551 2.197233  3.251061e-01  2.333865 1.735398  0.381711
   0.1     2  6.798270 5.365850  0.390419  2.707075 2.070347  4.160104e-01  2.172536 1.607752  0.444997
   0.1     3  6.666620 5.314722  0.383173  2.708571 2.063521  4.205775e-01  1.949039 1.490537  0.485578
   0.1     4  6.933027 5.450880  0.396277  2.729019 2.108753  4.406451e-01  2.119327 1.590028  0.429482
   0.1     5  6.658870 5.228377  0.426075  2.673850 2.030723  4.736155e-01  2.083414 1.585610  0.465514
   0.1     6  6.871475 5.373464  0.426460  2.727854 2.087930  4.336336e-01  2.084930 1.593528  0.457059
   0.1     7  6.927178 5.478382  0.371232  2.835678 2.182043  3.844511e-01  2.010927 1.545726  0.469801
   0.1     8  7.053830 5.368222  0.399044  2.760321 2.133928  3.958686e-01  2.190971 1.648751  0.424726
   0.1     9  6.615810 5.252569  0.373370  2.658875 2.081842  3.

In [12]:
import os
import numpy as np
import pandas as pd
from joblib import Parallel, delayed
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.impute import SimpleImputer

# ─── Configuration ─────────────────────────────────────────────────────────
FOLDS_DIR = "cv_folds"
FEATURE_COLS = [
    "PTS_8_game_avg", "PTS_8_game_slope",
    "REB_8_game_avg", "REB_8_game_slope",
    "AST_8_game_avg", "AST_8_game_slope",
    "opp_E_OFF_RATING", "opp_E_DEF_RATING",
    "opp_E_NET_RATING", "opp_REB_PCT"
]
TARGET_COLS = ["PTS", "REB", "AST"]
N_JOBS = 4

# ─── Hyperparameter grid ──────────────────────────────────────────────────
param_grid = {
    "alpha": [0.1, 1.0, 10.0]
}

# ─── Single-fold runner (returns per-target metrics) ───────────────────────
def run_lasso_fold(fold_idx, alpha):
    train_df = pd.read_csv(f"{FOLDS_DIR}/fold_{fold_idx}_train.csv", parse_dates=["GAME_DATE"])
    test_df  = pd.read_csv(f"{FOLDS_DIR}/fold_{fold_idx}_test.csv", parse_dates=["GAME_DATE"])

    X_train = train_df[FEATURE_COLS].values
    y_train = train_df[TARGET_COLS].values
    X_test  = test_df[FEATURE_COLS].values
    y_test  = test_df[TARGET_COLS].values

    imputer = SimpleImputer(strategy="mean")
    X_train = imputer.fit_transform(X_train)
    X_test = imputer.transform(X_test)

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Fit separate Lasso model for each target
    metrics = {"filters": None, "kernel_size": None, "epochs": None, "fold": fold_idx}
    for i, target in enumerate(TARGET_COLS):
        model = Lasso(alpha=alpha, max_iter=10000)
        model.fit(X_train, y_train[:, i])
        preds = model.predict(X_test)

        metrics[f"{target}_rmse"] = np.sqrt(mean_squared_error(y_test[:, i], preds))
        metrics[f"{target}_mae"]  = mean_absolute_error(y_test[:, i], preds)
        metrics[f"{target}_r2"]   = r2_score(y_test[:, i], preds)
    return metrics

# ─── Run across all folds and alphas ────────────────────────────────────────
all_results = []
for alpha in param_grid["alpha"]:
    fold_metrics = Parallel(n_jobs=N_JOBS)(
        delayed(run_lasso_fold)(fold, alpha) for fold in range(1, 11)
    )
    all_results.extend(fold_metrics)

# ─── Compile and display results ────────────────────────────────────────────
final_df = pd.DataFrame(all_results)
# Ensure column order matches CNN format
ordered_cols = [
    "filters", "kernel_size", "epochs", "fold",
    "PTS_rmse", "PTS_mae", "PTS_r2",
    "REB_rmse", "REB_mae", "REB_r2",
    "AST_rmse", "AST_mae", "AST_r2"
]
final_df = final_df[ordered_cols]
print(final_df.to_string(index=False))


filters kernel_size epochs  fold  PTS_rmse  PTS_mae    PTS_r2  REB_rmse  REB_mae        REB_r2  AST_rmse  AST_mae    AST_r2
   None        None   None     1  7.854028 6.140139  0.253482  2.792551 2.197233  3.251061e-01  2.333865 1.735398  0.381711
   None        None   None     2  6.798270 5.365850  0.390419  2.707075 2.070347  4.160104e-01  2.172536 1.607752  0.444997
   None        None   None     3  6.666620 5.314722  0.383173  2.708571 2.063521  4.205775e-01  1.949039 1.490537  0.485578
   None        None   None     4  6.933027 5.450880  0.396277  2.729019 2.108753  4.406451e-01  2.119327 1.590028  0.429482
   None        None   None     5  6.658870 5.228377  0.426075  2.673850 2.030723  4.736155e-01  2.083414 1.585610  0.465514
   None        None   None     6  6.871475 5.373464  0.426460  2.727854 2.087930  4.336336e-01  2.084930 1.593528  0.457059
   None        None   None     7  6.927178 5.478382  0.371232  2.835678 2.182043  3.844511e-01  2.010927 1.545726  0.469801
   None 